## CONSULTAS NO DATASET

In [1]:
%defaultDatasource jdbc:h2:mem:db

In [2]:
DROP TABLE IF EXISTS Jogador;

CREATE TABLE Jogador (
  nome VARCHAR(50) NOT NULL,
  data_de_nascimento INTEGER NOT NULL,
  altura INT,
  mao_dominante VARCHAR(1),
  local VARCHAR(3),
  PRIMARY KEY(nome, data_de_nascimento)
) AS SELECT
  name,date_of_birth,height,hand,country
FROM CSVREAD('../data/processed/men_players.csv');

In [3]:
SELECT * FROM Jogador

In [4]:
DROP TABLE IF EXISTS Torneio;

CREATE TABLE Torneio (
  location VARCHAR(100),
  surface VARCHAR(50),
  year INTEGER,
  tournament VARCHAR(100),
  country VARCHAR(5),
    
  PRIMARY KEY(tournament, year),
) AS SELECT
  Location,Surface,Year,Tournament, Country
FROM CSVREAD('../data/processed/tournaments.csv');

In [5]:
SELECT * FROM Torneio

In [6]:
DROP TABLE IF EXISTS Confronto;

CREATE TABLE Confronto (
  winner VARCHAR(100) NOT NULL,
  loser VARCHAR(100) NOT NULL,
  round VARCHAR(100) NOT NULL,
  tournament VARCHAR(200),
  data VARCHAR(50),
  oddwinner FLOAT,
  oddloser FLOAT,
  setwinner FLOAT,
  setloser FLOAT,
  year INTEGER,
  PRIMARY KEY(winner,loser,data),
  FOREIGN KEY(winner) REFERENCES Jogador(nome),
  FOREIGN KEY(loser) REFERENCES Jogador(nome),
  FOREIGN KEY(tournament, year) REFERENCES Torneio(tournament, year)
) AS SELECT
  Winner, Loser, Round, Tournament, Data, AvgW, AvgL,Wsets, Lsets, Year
FROM CSVREAD('../data/processed/confrontations_since_2010.csv');

In [7]:
SELECT * FROM Confronto

In [8]:
DROP TABLE IF EXISTS Historico;

CREATE TABLE Historico (
  name VARCHAR(100) NOT NULL,
  followers_2019 INTEGER,
  followers_2021 INTEGER,
  ano_2010 INTEGER,
  ano_2011 INTEGER,
  ano_2012 INTEGER,
  ano_2013 INTEGER,
  ano_2014 INTEGER,
  ano_2015 INTEGER,
  ano_2016 INTEGER,
  ano_2017 INTEGER,
  ano_2018 INTEGER,
  ano_2019 INTEGER,
  ano_2020 INTEGER,
  PRIMARY KEY(name),
  FOREIGN KEY(name) REFERENCES Jogador(nome),
) AS SELECT
  name,followers_2019,followers_2021,ano_2010,ano_2011,ano_2012,ano_2013,ano_2014,ano_2015,ano_2016,ano_2017,ano_2018,ano_2019,ano_2020
FROM CSVREAD('../data/processed/history.csv');


In [9]:
SELECT * FROM Historico

## Quais os jogadores canhotos brasileiros?

In [10]:
SELECT nome FROM Jogador
WHERE mao_dominante = 'L'
AND local = 'BRA'

In [11]:
SELECT mao_dominante, COUNT(*) num 
FROM Jogador  
WHERE local = 'BRA'
GROUP BY mao_dominante

## Qual a distribuição do tipo de superfície nos torneios?

In [12]:
SELECT surface, COUNT(*) num 
FROM Torneio  
GROUP BY surface

## Quais são os jogadores mais populares?

In [13]:
SELECT NAME, FOLLOWERS_2021
FROM HISTORICO
ORDER BY FOLLOWERS_2021 DESC

## Quais os campeonatos mais jogados pelos 3 jogadores mais famosos?

In [14]:
DROP VIEW IF EXISTS TOP3;
CREATE VIEW TOP3 AS 
SELECT * 
FROM Confronto
WHERE winner = 'Nadal R.'
OR winner = 'Federer R.'
OR winner = 'Djokovic N.'

In [15]:
select * from top3

In [16]:
SELECT winner, TOP3.tournament, count(*)num 
FROM TOP3
GROUP BY TOP3.tournament, winner
ORDER BY num desc

## Como a altura de um jogador influencia nas vitórias?

In [17]:
SELECT MAX(altura)
FROM Jogador

211

In [18]:
SELECT *
FROM Jogador
WHERE altura = (SELECT MIN(altura) FROM Jogador)
OR altura = SELECT MAX(altura) FROM Jogador

In [19]:
SELECT WINNER, COUNT(*) NUM
FROM CONFRONTO
WHERE WINNER = 'Rochus O.'
OR WINNER = 'Opelka R.'
GROUP BY WINNER

In [20]:
SELECT LOSER, COUNT(*) NUM
FROM CONFRONTO
WHERE LOSER = 'Rochus O.'
OR LOSER = 'Opelka R.'
GROUP BY LOSER

## QUAIS AS MELHORES PERFORMANCES DE JOGADORES EM CADA  ANO NOS ÚLTIMOS 10 ANOS?

In [21]:
SELECT *
FROM Historico
WHERE ano_2010 =  (SELECT MAX(ano_2010) FROM Historico) OR
ano_2011 =  (SELECT MAX(ano_2011) FROM Historico) OR
ano_2012 =  (SELECT MAX(ano_2012) FROM Historico) OR
ano_2013 =  (SELECT MAX(ano_2013) FROM Historico) OR
ano_2014 =  (SELECT MAX(ano_2014) FROM Historico) OR
ano_2015 =  (SELECT MAX(ano_2015) FROM Historico) OR
ano_2016 =  (SELECT MAX(ano_2016) FROM Historico) OR
ano_2017 =  (SELECT MAX(ano_2017) FROM Historico) OR
ano_2018 =  (SELECT MAX(ano_2018) FROM Historico) OR
ano_2019 =  (SELECT MAX(ano_2019) FROM Historico) OR
ano_2020 =  (SELECT MAX(ano_2020) FROM Historico) 

## QUANTAS PARTIDAS OS MELHORES JOGADORES DA DÉCADA GANHARAM NO ANO DO SEU AUGE?

In [22]:
SELECT WINNER, COUNT(*) Vitórias
FROM CONFRONTO, Historico
WHERE 
(winner = name and year =  2013 and ano_2013 = (SELECT MAX(ano_2013) FROM Historico))
OR
(winner = name and year =  2015 and ano_2015 = (SELECT MAX(ano_2015) FROM Historico))
OR
(winner = name and year =  2016 and ano_2016 = (SELECT MAX(ano_2016) FROM Historico))
GROUP BY WINNER

## QUAIS PAÍSES POSSUEM MAIS PONTOS PER CAPITA NO RANKING EM 2019?

In [23]:
DROP VIEW IF EXISTS Ranking_per_country;
CREATE VIEW Ranking_per_country AS
SELECT nome, ano_2019, local
FROM Jogador, Historico
WHERE nome = name

In [24]:
SELECT LOCAL, SUM(ANO_2019) total
FROM Ranking_per_country
GROUP BY LOCAL
ORDER BY total desc

In [25]:
DROP VIEW IF EXISTS Ranking_per_capita;
CREATE VIEW Ranking_per_capita as
SELECT LOCAL, SUM(ANO_2019) total, Count(local) num
FROM Ranking_per_country
GROUP BY LOCAL
ORDER BY total desc

In [26]:
SELECT local, total/num as pontos_por_jogador
from Ranking_per_capita
ORDER BY pontos_por_jogador desc

## Quais foram os maiores enganadores nas partidas?

### Quem foi subestimado?

In [42]:
DROP VIEW IF EXISTS WSUBESTIMADO;
CREATE VIEW WSUBESTIMADO AS
SELECT winner, AVG(oddwinner) odd_media, Count(winner) vitorias
FROM CONFRONTO
GROUP BY WINNER
ORDER BY odd_media desc

In [62]:
DROP VIEW IF EXISTS LSUBESTIMADO;
CREATE VIEW LSUBESTIMADO AS
SELECT loser, AVG(oddloser) odd_media, Count(loser) derrotas
FROM CONFRONTO
GROUP BY LOSER
ORDER BY odd_media desc

In [63]:
SELECT winner, (wsubestimado.odd_media * (VITORIAS/DERROTAS)) as enganação
FROM LSUBESTIMADO, WSUBESTIMADO
WHERE WINNER = LOSER
ORDER BY ENGANAÇÃO DESC



### QUEM FOI SUPERESTIMADO?

In [65]:
SELECT LOSER, ((DERROTAS/VITORIAS)/lsubestimado.odd_media) as enganação
FROM LSUBESTIMADO, WSUBESTIMADO
WHERE WINNER = LOSER
ORDER BY ENGANAÇÃO DESC
